In [168]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.base import BaseEstimator
from sklearn.metrics import ndcg_score

# Time series decomposition
from statsmodels.tsa.seasonal import seasonal_decompose

# Chart drawing
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Mute sklearn warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

# Show charts when running kernel
init_notebook_mode(connected=True)

# Change default background color for all visualizations
layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(250,250,250,0.8)')
fig = go.Figure(layout=layout)
templated_fig = pio.to_templated(fig)
pio.templates['my_template'] = templated_fig.layout.template
pio.templates.default = 'my_template'

In [169]:

file_path = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/scripts/dati_with_context_f.csv' # cleaned data
df = pd.read_csv(file_path)
df = df.dropna(subset=['relative_quarterly_return'])

df.info()
df.tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11723 entries, 1 to 11812
Data columns (total 64 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   tic                             11723 non-null  object 
 1   datadate                        11723 non-null  object 
 2   roa                             11692 non-null  float64
 3   roe                             11563 non-null  float64
 4   pe_inc                          11199 non-null  float64
 5   divyield                        9443 non-null   float64
 6   pcf                             11692 non-null  float64
 7   roce                            11586 non-null  float64
 8   gpm                             11711 non-null  float64
 9   evm                             11636 non-null  float64
 10  npm                             11712 non-null  float64
 11  PEG_trailing                    8110 non-null   float64
 12  de_ratio                        

,tic,datadate,roa,roe,pe_inc,divyield,pcf,roce,gpm,evm,...,r_roe,r_eg,divyield_Median,PEG_trailing_Median,pe_inc_Median,ptb_Median,evm_Median,roce_Median,roe_Median,bm_Median
11808,ZTS,2018-12-31,0.230,0.594,32.160,0.696,25.204,0.250,0.708,21.308,...,3.2636,4.3514,1.33,-0.101,-7.090,3.183,-4.273,-0.413,-0.531,0.227
11809,ZTS,2019-03-31,0.239,0.577,35.332,0.649,27.021,0.252,0.715,21.631,...,-3.5981,-5.1201,1.24,-0.079,-2.895,3.745,-2.919,-0.446,-0.452,0.308
11810,ZTS,2019-06-30,0.233,0.539,44.671,0.519,32.794,0.241,0.722,22.510,...,2.6492,1.1047,1.21,-0.115,-7.734,3.796,-3.920,-0.411,-0.512,0.251
11811,ZTS,2019-09-30,0.228,0.538,39.907,0.544,31.944,0.234,0.728,23.287,...,3.8375,0.8189,1.22,-0.092,-6.526,3.278,-3.883,-0.416,-0.509,0.258
11812,ZTS,2019-12-31,0.226,0.517,42.839,0.600,35.251,0.235,0.736,27.557,...,-1.9816,0.6485,1.10,-0.088,-8.014,3.910,-3.433,-0.447,-0.545,0.285


In [170]:
print(df.columns.tolist())
print(df.notna().sum().sort_values())



['tic', 'datadate', 'roa', 'roe', 'pe_inc', 'divyield', 'pcf', 'roce', 'gpm', 'evm', 'npm', 'PEG_trailing', 'de_ratio', 'ptb', 'market_cap', 'prccq_change', 'capxy_change', 'oancfy_change', 'actq_change', 'lctq_change', 'roa_change', 'roe_change', 'pe_inc_change', 'divyield_change', 'pcf_change', 'roce_change', 'gpm_change', 'evm_change', 'npm_change', 'PEG_trailing_change', 'de_ratio_change', 'ptb_change', 'next_prccq_change', 'relative_quarterly_return', 'next_relative_quarterly_return', 'volatility_score', 'FFI10_desc', 'mktrf', 'smb', 'hml', 'rmw', 'cma', 'rf', 'umd', 'h_1_x', 'h_3_x', 'h_12_x', 'h_1_y', 'h_3_y', 'h_12_y', 'r_f', 'r_mkt', 'r_me', 'r_ia', 'r_roe', 'r_eg', 'divyield_Median', 'PEG_trailing_Median', 'pe_inc_Median', 'ptb_Median', 'evm_Median', 'roce_Median', 'roe_Median', 'bm_Median']
PEG_trailing_change     7371
PEG_trailing            8110
oancfy_change           8131
capxy_change            8602
lctq_change             9189
                       ...  
roe_Median   

In [171]:


df = df.loc[:, df.count() >= 11000]

# Drop rows with missing values
df = df.dropna()

# Display dataframe info
df.info()

print(df.columns.tolist())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10509 entries, 2 to 11812
Data columns (total 56 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   tic                             10509 non-null  object 
 1   datadate                        10509 non-null  object 
 2   roa                             10509 non-null  float64
 3   roe                             10509 non-null  float64
 4   pe_inc                          10509 non-null  float64
 5   pcf                             10509 non-null  float64
 6   roce                            10509 non-null  float64
 7   gpm                             10509 non-null  float64
 8   evm                             10509 non-null  float64
 9   npm                             10509 non-null  float64
 10  de_ratio                        10509 non-null  float64
 11  ptb                             10509 non-null  float64
 12  market_cap                      

In [172]:
# Find the unique number of stocks in the dataframe
unique_gvkeys = df['tic'].unique()

# Print the result
print(f"Number of unique gvkeys: {len(unique_gvkeys)}")
print(unique_gvkeys)


Number of unique gvkeys: 76
['AAPL' 'ABT' 'ADM' 'AFL' 'AIG' 'AMD' 'AMT' 'AMZN' 'APD' 'BA' 'BAX' 'BMY'
 'BRK.B' 'CAT' 'CL' 'CME' 'COP' 'CTSH' 'CVS' 'CVX' 'DIS' 'ECL' 'EQIX'
 'EXC' 'EXPE' 'F' 'FIS' 'GE' 'GOOGL' 'GS' 'HCA' 'IBM' 'INTC' 'JNJ' 'JPM'
 'KMB' 'KO' 'LLY' 'LMT' 'LUV' 'MA' 'MCD' 'META' 'MMC' 'MMM' 'MRK' 'MS'
 'MSFT' 'NFLX' 'NOC' 'OXY' 'PFE' 'PG' 'PGR' 'PPL' 'PYPL' 'QCOM' 'REGN'
 'RJF' 'RTX' 'SBUX' 'SCHW' 'SPGI' 'T' 'TMO' 'TSLA' 'TSN' 'UNH' 'UPS' 'V'
 'VZ' 'WBA' 'WDC' 'WFC' 'XOM' 'ZTS']


In [173]:
df = df.dropna()
df = df.sort_values(by=['datadate'])

unique_gvkeys = df['tic'].unique()

# Print the result
print(f"Number of unique gvkeys: {len(unique_gvkeys)}")

df.tail()

Number of unique gvkeys: 76


,tic,datadate,roa,roe,pe_inc,pcf,roce,gpm,evm,npm,...,r_roe,r_eg,divyield_Median,PEG_trailing_Median,pe_inc_Median,ptb_Median,evm_Median,roce_Median,roe_Median,bm_Median
876,AIG,2019-12-31,0.023,0.054,11.273,-37.158,0.066,0.236,7.292,0.067,...,-1.9816,0.6485,2.2,0.616,13.611,1.387,10.016,0.102,0.097,0.770
5972,JPM,2019-12-31,0.027,0.153,10.831,59.034,0.083,0.773,13.715,0.256,...,-1.9816,0.6485,2.2,0.616,13.611,1.387,10.016,0.102,0.097,0.770
5772,JNJ,2019-12-31,0.181,0.228,23.886,15.131,0.237,0.751,14.671,0.184,...,-1.9816,0.6485,1.1,-0.088,-8.014,3.910,-3.433,-0.447,-0.545,0.285
1932,BAX,2019-12-31,0.161,0.125,43.249,20.255,0.154,0.502,17.713,0.088,...,-1.9816,0.6485,2.2,0.616,13.611,1.387,10.016,0.102,0.097,0.770
11812,ZTS,2019-12-31,0.226,0.517,42.839,35.251,0.235,0.736,27.557,0.239,...,-1.9816,0.6485,1.1,-0.088,-8.014,3.910,-3.433,-0.447,-0.545,0.285


In [174]:
tickers_to_drop = ['AMT', 'CTSH', 'GOOGL', 'META', 'PYPL', 'RJF', 'TSLA', 'ZTS']

#df = df[~df['tic'].isin(tickers_to_drop)]

In [175]:
train_start_date = '1900-03-31'
valid_start_date = '2010-03-31'
test_start_date = '2013-03-31'
end_date = '2026-09-30'

df = df.sort_values(by=['datadate'])

train_df = df[(df['datadate'] >= train_start_date) & (df['datadate'] < valid_start_date)]
valid_df = df[(df['datadate'] >= valid_start_date) & (df['datadate'] < test_start_date)]
test_df = df[(df['datadate'] >= test_start_date) & (df['datadate'] < end_date)]

test_df = test_df.sort_values(by=['datadate', 'next_prccq_change'], ascending=[True, False])
test_df.tail()


,tic,datadate,roa,roe,pe_inc,pcf,roce,gpm,evm,npm,...,r_roe,r_eg,divyield_Median,PEG_trailing_Median,pe_inc_Median,ptb_Median,evm_Median,roce_Median,roe_Median,bm_Median
4330,EXPE,2019-12-31,0.095,0.138,26.159,4.793,0.111,0.828,11.237,0.047,...,-1.9816,0.6485,1.37,0.306,1.471,3.459,12.578,0.043,0.004,0.289
4530,F,2019-12-31,0.060,0.001,306.600,1.537,0.030,0.180,12.526,0.000,...,-1.9816,0.6485,2.31,0.416,11.098,2.718,9.116,0.098,0.085,0.413
2953,COP,2019-12-31,0.190,0.188,7.566,4.714,0.130,0.444,6.459,0.221,...,-1.9816,0.6485,2.55,0.090,3.964,0.919,5.440,0.061,0.047,1.103
876,AIG,2019-12-31,0.023,0.054,11.273,-37.158,0.066,0.236,7.292,0.067,...,-1.9816,0.6485,2.20,0.616,13.611,1.387,10.016,0.102,0.097,0.770
8520,OXY,2019-12-31,0.129,-0.023,-26.836,3.974,0.081,0.540,8.755,-0.032,...,-1.9816,0.6485,2.55,0.090,3.964,0.919,5.440,0.061,0.047,1.103


In [176]:
train_group = list(train_df['datadate'].value_counts().sort_index().tolist())
valid_group = list(valid_df['datadate'].value_counts().sort_index().tolist())
test_group = list(test_df['datadate'].value_counts().sort_index().tolist())

print(train_group)

[4, 24, 30, 32, 32, 31, 33, 34, 35, 32, 32, 37, 36, 35, 37, 38, 35, 37, 33, 33, 32, 36, 26, 26, 28, 37, 35, 34, 34, 37, 36, 36, 35, 38, 38, 38, 38, 40, 39, 39, 37, 36, 30, 32, 32, 35, 34, 39, 43, 39, 39, 44, 43, 41, 42, 46, 46, 43, 43, 49, 50, 50, 50, 50, 50, 51, 54, 52, 52, 48, 47, 52, 51, 50, 51, 52, 52, 50, 51, 52, 52, 50, 51, 53, 53, 52, 51, 52, 51, 53, 53, 51, 52, 53, 53, 54, 54, 52, 54, 54, 55, 55, 56, 56, 56, 57, 56, 57, 57, 56, 56, 57, 57, 59, 60, 59, 59, 58, 59, 60, 60, 59, 59, 59, 61, 61, 60, 61, 61, 60, 61, 63, 63, 63, 63, 63, 63, 63, 64, 63, 63, 64, 64, 65, 66, 66, 66, 66, 66, 66, 66, 67, 66, 63, 67, 67, 67, 64]


In [177]:
# Create a copy of the test results
results = test_df[['datadate', 'tic', 'next_prccq_change', 'next_relative_quarterly_return']].copy()

# Define columns to drop
drop_cols = ['datadate', 'tic', 'next_prccq_change', 'FFI10_desc']

# Drop the specified columns from train, validation, and test datasets
train_df = train_df.drop(columns=drop_cols)
valid_df = valid_df.drop(columns=drop_cols)
test_df  = test_df.drop(columns=drop_cols)

# Display the first few rows of the training dataset
train_df.head()


,roa,roe,pe_inc,pcf,roce,gpm,evm,npm,de_ratio,ptb,...,r_roe,r_eg,divyield_Median,PEG_trailing_Median,pe_inc_Median,ptb_Median,evm_Median,roce_Median,roe_Median,bm_Median
1335,0.145,0.093,17.928,8.009,0.117,0.507,7.570,0.057,1.440,1.387,...,-6.2954,-10.1619,3.51,0.908,14.712,1.435,9.151,0.102,0.087,0.612
3443,0.204,0.115,41.576,41.821,0.209,0.309,13.130,0.130,0.227,3.479,...,-6.2954,-10.1619,2.95,0.855,12.418,1.368,7.375,0.187,0.106,0.655
11213,0.112,0.094,14.020,16.465,0.122,0.276,7.442,0.013,1.215,1.280,...,-6.2954,-10.1619,3.70,1.084,12.710,1.218,7.059,0.166,0.091,0.689
7019,0.248,0.441,30.332,25.417,0.283,0.386,16.650,0.091,1.872,9.436,...,-6.2954,-10.1619,2.95,0.855,12.418,1.368,7.375,0.187,0.106,0.655
3444,0.204,0.105,47.375,56.869,0.209,0.309,13.130,0.135,0.227,4.730,...,5.6547,6.2280,2.71,1.046,13.920,1.415,7.319,0.183,0.115,0.673


In [178]:
# Define target variable (y) and features (X) for train, validation, and test sets
y_train = train_df['next_relative_quarterly_return'].copy()
X_train = train_df.drop(columns=['next_relative_quarterly_return'])

y_valid = valid_df['next_relative_quarterly_return'].copy()
X_valid = valid_df.drop(columns=['next_relative_quarterly_return'])

y_test  = test_df['next_relative_quarterly_return'].copy()
X_test  = test_df.drop(columns=['next_relative_quarterly_return'])

# Display the first few rows of X_train
X_train.head()


,roa,roe,pe_inc,pcf,roce,gpm,evm,npm,de_ratio,ptb,...,r_roe,r_eg,divyield_Median,PEG_trailing_Median,pe_inc_Median,ptb_Median,evm_Median,roce_Median,roe_Median,bm_Median
1335,0.145,0.093,17.928,8.009,0.117,0.507,7.570,0.057,1.440,1.387,...,-6.2954,-10.1619,3.51,0.908,14.712,1.435,9.151,0.102,0.087,0.612
3443,0.204,0.115,41.576,41.821,0.209,0.309,13.130,0.130,0.227,3.479,...,-6.2954,-10.1619,2.95,0.855,12.418,1.368,7.375,0.187,0.106,0.655
11213,0.112,0.094,14.020,16.465,0.122,0.276,7.442,0.013,1.215,1.280,...,-6.2954,-10.1619,3.70,1.084,12.710,1.218,7.059,0.166,0.091,0.689
7019,0.248,0.441,30.332,25.417,0.283,0.386,16.650,0.091,1.872,9.436,...,-6.2954,-10.1619,2.95,0.855,12.418,1.368,7.375,0.187,0.106,0.655
3444,0.204,0.105,47.375,56.869,0.209,0.309,13.130,0.135,0.227,4.730,...,5.6547,6.2280,2.71,1.046,13.920,1.415,7.319,0.183,0.115,0.673


In [179]:
# Scaling
scaler = MinMaxScaler()

# Fit the scaler on the training/validation data
X_train_scaled = scaler.fit_transform(X_train)

# Now apply the same scaling to the val/test data using the already fitted scaler
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# Convert the scaled data back to DataFrame for consistency
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_valid_scaled = pd.DataFrame(X_valid_scaled, columns=X_valid.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

#X_train_scaled.info()

print(X_train_scaled.columns.tolist())


['roa', 'roe', 'pe_inc', 'pcf', 'roce', 'gpm', 'evm', 'npm', 'de_ratio', 'ptb', 'market_cap', 'prccq_change', 'roa_change', 'roe_change', 'pe_inc_change', 'pcf_change', 'roce_change', 'gpm_change', 'evm_change', 'npm_change', 'de_ratio_change', 'ptb_change', 'relative_quarterly_return', 'volatility_score', 'mktrf', 'smb', 'hml', 'rmw', 'cma', 'rf', 'umd', 'h_1_x', 'h_3_x', 'h_12_x', 'h_1_y', 'h_3_y', 'h_12_y', 'r_f', 'r_mkt', 'r_me', 'r_ia', 'r_roe', 'r_eg', 'divyield_Median', 'PEG_trailing_Median', 'pe_inc_Median', 'ptb_Median', 'evm_Median', 'roce_Median', 'roe_Median', 'bm_Median']


In [180]:

context = ['relative_quarterly_return_-1', 'relative_quarterly_return_-2', 'relative_quarterly_return_-3', 'mktrf', 'smb', 'hml', 'rmw', 'cma', 'rf', 'umd', 'h_1_x', 'h_3_x', 'h_12_x', 'h_1_y', 'h_3_y', 'h_12_y', 'r_f', 'r_mkt', 'r_me', 'r_ia', 'r_roe', 'r_eg', 'indret_ew', 'CAPEI_Median', 'bm_Median', 'evm_Median', 'pe_inc_Median', 'ptb_Median', 'PEG_trailing_Median', 'divyield_Median', 'roa_Median', 'roce_Median', 'roe_Median', 'gpm_Median', 'npm_Median', 'opmad_Median', 'de_ratio_Median', 'debt_ebitda_Median', 'debt_assets_Median', 'intcov_ratio_Median', 'cash_ratio_Median', 'curr_ratio_Median', 'quick_ratio_Median', 'at_turn_Median', 'inv_turn_Median', 'rect_turn_Median']
context = ['market_cap', 'volatility_score', 'mktrf', 'smb', 'hml', 'rmw', 'cma', 'rf', 'umd', 'h_1_x', 'h_3_x', 'h_12_x', 'h_1_y', 'h_3_y', 'h_12_y', 'r_f', 'r_mkt', 'r_me', 'r_ia', 'r_roe', 'r_eg', 'divyield_Median', 'PEG_trailing_Median', 'pe_inc_Median', 'ptb_Median', 'evm_Median', 'roce_Median', 'roe_Median', 'bm_Median']


C_train = X_train_scaled[context].to_numpy()
C_valid = X_valid_scaled[context].to_numpy()
C_test = X_test_scaled[context].to_numpy()

X_train_context = X_train_scaled.drop(columns=context)
X_valid_context = X_valid_scaled.drop(columns=context)
X_test_context = X_test_scaled.drop(columns=context)



In [181]:


%%time

parameters = {
    'n_estimators': [300],
    'learning_rate': [0.1],
    'max_depth': [8],
    'gamma': [0.1],
    'random_state': [42]
}

eval_set = [(X_train_scaled, y_train), (X_valid_scaled, y_valid)]
model = xgb.XGBRegressor(eval_set=eval_set, objective='reg:squarederror', verbose=False)
clf = GridSearchCV(model, parameters)

clf.fit(X_train_scaled, y_train)

print(f'Best params: {clf.best_params_}')
print(f'Best validation score = {clf.best_score_}')



/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning:

[00:18:22] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "eval_set", "verbose" } are not used.




/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning:

[00:18:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "eval_set", "verbose" } are not used.


/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning:

[00:18:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "eval_set", "verbose" } are not used.


/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning:

[00:18:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "eval_set", "verbose" } are not used.


/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning:

[00:18:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "eval_set", "verbose" } are not used.


/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning:

[00:18:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { 

Best params: {'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 300, 'random_state': 42}
Best validation score = 0.20207122569669464
CPU times: user 21.4 s, sys: 869 ms, total: 22.3 s
Wall time: 4.01 s


In [182]:
model = xgb.XGBRegressor(**clf.best_params_, objective='reg:squarederror')
model.fit(X_train_scaled, y_train, eval_set=eval_set, verbose=False)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [183]:
# **Define Model Parameters**
ranker_model = xgb.XGBRanker(
    objective="rank:ndcg",
    eval_metric="ndcg@10",
    ndcg_exp_gain=False,
    learning_rate=0.1,
    max_depth=8,
    gamma=0.1,  
    random_state=42,
    n_estimators=300
)

ranker_model.fit(
        X_train_scaled,
        y_train,
        group=train_group,
        verbose=True,
    )


XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric='ndcg@10', feature_types=None, gamma=0.1,
          grow_policy=None, importance_type=None, interaction_constraints=None,
          learning_rate=0.1, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=300,
          n_jobs=None, ndcg_exp_gain=False, num_parallel_tree=None, ...)

In [184]:
y_train_pred1 = model.predict(X_train_scaled)
y_valid_pred1 = model.predict(X_valid_scaled)
y_test_pred1 = model.predict(X_test_scaled)

# Classifier
y_train_pred_binary1 = (y_train_pred1 >= 0).astype(int)
y_valid_pred_binary1 = (y_valid_pred1 >= 0).astype(int)
y_test_pred_binary1 = (y_test_pred1 >= 0).astype(int)

y_train_binary1 = (y_train >= 0).astype(int)
y_valid_binary1 = (y_valid >= 0).astype(int)
y_test_binary1 = (y_test >= 0).astype(int)

print(classification_report(y_test_binary1, y_test_pred_binary1))

y_train_pred2 = ranker_model.predict(X_train_scaled)
y_valid_pred2 = ranker_model.predict(X_valid_scaled)
y_test_pred2 = ranker_model.predict(X_test_scaled)

# Classifier
y_train_pred_binary2 = (y_train_pred1 >= 0).astype(int)
y_valid_pred_binary2 = (y_valid_pred1 >= 0).astype(int)
y_test_pred_binary2 = (y_test_pred1 >= 0).astype(int)

y_train_binary2 = (y_train >= 0).astype(int)
y_valid_binary2 = (y_valid >= 0).astype(int)
y_test_binary2 = (y_test >= 0).astype(int)

print(classification_report(y_test_binary2, y_test_pred_binary2))



              precision    recall  f1-score   support

           0       0.73      0.57      0.64       954
           1       0.66      0.80      0.72       986

    accuracy                           0.69      1940
   macro avg       0.69      0.68      0.68      1940
weighted avg       0.69      0.69      0.68      1940

              precision    recall  f1-score   support

           0       0.73      0.57      0.64       954
           1       0.66      0.80      0.72       986

    accuracy                           0.69      1940
   macro avg       0.69      0.68      0.68      1940
weighted avg       0.69      0.69      0.68      1940



In [185]:
results['y_pred1'] = y_test_pred1
results['y_pred2'] = y_test_pred2
results1 = results.sort_values(by=['datadate', 'y_pred1'], ascending=[True, False])

results1.head(10)



,datadate,tic,next_prccq_change,next_relative_quarterly_return,y_pred1,y_pred2
1037,2013-03-31,AMD,0.600000,0.566207,0.319536,1.511801
329,2013-03-31,ABT,-0.012458,-0.046250,0.215000,1.660079
9481,2013-03-31,REGN,0.274808,0.241016,0.202543,1.222047
8097,2013-03-31,NFLX,0.115226,0.081433,0.167186,0.216506
849,2013-03-31,AIG,0.151468,0.117676,0.116388,0.189624
7221,2013-03-31,META,-0.027365,-0.061158,0.107656,-1.500196
11356,2013-03-31,WDC,0.234639,0.200846,0.095437,1.322385
681,2013-03-31,AFL,0.117263,0.083470,0.094977,0.194564
6744,2013-03-31,LMT,0.123705,0.089912,0.091968,0.497474
2257,2013-03-31,BRK.B,0.074088,0.040296,0.090515,-0.460894


In [189]:
def softmax_weighted_return(df, sort_col, target_col, n=5, largest=True):
    if largest:
        selected = df.nlargest(n, sort_col)
    else:
        selected = df.nsmallest(n, sort_col)
    # Compute softmax weights from the predicted scores:
    e_scores = np.exp(selected[sort_col])
    weights = e_scores / e_scores.sum()
    return (selected[target_col] * weights).sum()

def exponential_weighted_return(df, sort_col, target_col, n=20, largest=True, alpha=1):
    if largest:
        selected = df.nlargest(n, sort_col)
    else:
        selected = df.nsmallest(n, sort_col)
    selected = selected.sort_values(by=sort_col, ascending=not largest).copy()
    selected['rank'] = range(1, len(selected) + 1)
    # Exponential weights: weight = exp(-alpha * (rank - 1))
    selected['weight'] = np.exp(-alpha * (selected['rank'] - 1))
    selected['weight'] /= selected['weight'].sum()  # Normalize weights
    return (selected[target_col] * selected['weight']).sum()


# Baseline aggregation remains unchanged.
df_avg = results.groupby('datadate').agg(
    stocks=('next_prccq_change', 'size'),
    baseline=('next_prccq_change', 'mean')
).reset_index()

# For the predicted top 10, weight by predicted returns ('pred_rank_score')
df_avg['top_10_xgb'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='y_pred1',
        target_col='next_prccq_change',
        largest=True
    )
).reset_index(level=0, drop=True)

# For the predicted top 10, weight by predicted returns ('pred_rank_score')
df_avg['top_10_rank'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='y_pred2',
        target_col='next_prccq_change',
        largest=True
    )
).reset_index(level=0, drop=True)

# For the ideal top 10, weight by the actual returns
df_avg['top_10_ideal'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='next_prccq_change',
        target_col='next_prccq_change',
        largest=True
    )
).reset_index(level=0, drop=True)

# For the predicted bottom 10, weight by predicted returns
df_avg['bottom_10_xgb'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='y_pred1',
        target_col='next_prccq_change',
        largest=False
    )
).reset_index(level=0, drop=True)

# For the predicted bottom 10, weight by predicted returns
df_avg['bottom_10_rank'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='y_pred2',
        target_col='next_prccq_change',
        largest=False
    )
).reset_index(level=0, drop=True)

# For the ideal bottom 10, weight by the actual returns
df_avg['bottom_10_ideal'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='next_prccq_change',
        target_col='next_prccq_change',
        largest=False
    )
).reset_index(level=0, drop=True)

# Display the first 100 rows of the new dataframe
df_avg.head(100)


,datadate,stocks,baseline,top_10_xgb,top_10_rank,top_10_ideal,bottom_10_xgb,bottom_10_rank,bottom_10_ideal
0,2013-03-31,70,0.077383,0.405624,0.163682,1.333543,-0.117949,-0.126518,-0.165985
1,2013-06-30,69,0.081841,0.287282,0.736588,0.887509,0.146116,0.018505,-0.112950
2,2013-09-30,70,0.105676,0.087735,0.310534,0.331823,-0.003792,-0.038681,-0.178453
3,2013-12-31,70,0.030107,0.248097,0.273782,0.348489,-0.022027,0.032746,-0.139108
4,2014-03-31,70,0.031233,0.052244,0.162001,0.236874,-0.240577,-0.322707,-0.346195
5,2014-06-30,72,0.021678,0.071698,0.106555,0.256985,-0.058344,-0.063554,-0.164261
6,2014-09-30,71,0.041867,0.121888,0.214745,0.239875,-0.191037,-0.134626,-0.224439
7,2014-12-31,70,0.013406,0.061940,0.148235,0.209248,0.009605,-0.108492,-0.165701
8,2015-03-31,71,0.018974,0.042695,0.391555,0.484331,-0.176878,-0.183199,-0.207809
9,2015-06-30,71,-0.057272,-0.016240,0.158197,0.175216,-0.267649,-0.396946,-0.431121


In [190]:
for portfolio in ['baseline', 'top_10_xgb', 'top_10_rank', 'bottom_10_xgb', 'bottom_10_rank']:

    mean_return = df_avg[portfolio].mean()

    # Calculate the standard deviation of the returns of the top 10 portfolio
    std_return = df_avg[portfolio].std()

    # Assuming the risk-free rate is 0 (if you have a specific risk-free rate, replace 0 with that value)
    risk_free_rate = 0.025

    # Calculate the Sharpe ratio
    sharpe_ratio = (np.abs(mean_return) - risk_free_rate) / std_return

    # Print the Sharpe ratio
    print(f"Mean quarterly return of the {portfolio} portfolio: {mean_return}")
    print(f"Sharpe Ratio of the {portfolio} portfolio: {sharpe_ratio}")
    print("")

Mean quarterly return of the baseline portfolio: 0.028449484738280097
Sharpe Ratio of the baseline portfolio: 0.05251906444681616

Mean quarterly return of the top_10_xgb portfolio: 0.09741008160147394
Sharpe Ratio of the top_10_xgb portfolio: 0.6456746804811893

Mean quarterly return of the top_10_rank portfolio: 0.23480367166612995
Sharpe Ratio of the top_10_rank portfolio: 1.3007894851331103

Mean quarterly return of the bottom_10_xgb portfolio: -0.10584181610837766
Sharpe Ratio of the bottom_10_xgb portfolio: 0.7408530585504215

Mean quarterly return of the bottom_10_rank portfolio: -0.1191476677361883
Sharpe Ratio of the bottom_10_rank portfolio: 0.8433157452014234



In [188]:
import plotly.graph_objects as go

# Assuming you have a dataframe 'df_avg' with columns 'datadate', 'baseline_portfolio', and 'top_10'

# Calculate the cumulative return for both portfolios
df_avg['baseline_cum_return'] = (1 + df_avg['baseline']).cumprod()
df_avg['top_10_xgb_cum_return'] = (1 + df_avg['top_10_xgb']).cumprod()
df_avg['top_10_rank_cum_return'] = (1 + df_avg['top_10_rank']).cumprod()

# Calculate the value over time of a $1,000 investment
initial_investment = 1000
df_avg['baseline_value'] = initial_investment * df_avg['baseline_cum_return']
df_avg['top_10_xgb_value'] = initial_investment * df_avg['top_10_xgb_cum_return']
df_avg['top_10_rank_value'] = initial_investment * df_avg['top_10_rank_cum_return']

# Create the plot
fig = go.Figure()

# Add baseline portfolio trace
fig.add_trace(go.Scatter(x=df_avg['datadate'], y=df_avg['baseline_value'],
                         mode='lines', name='Baseline Portfolio'))

# Add baseline portfolio trace
fig.add_trace(go.Scatter(x=df_avg['datadate'], y=df_avg['top_10_xgb_value'],
                         mode='lines', name='XGB Portfolio'))

# Add baseline portfolio trace
fig.add_trace(go.Scatter(x=df_avg['datadate'], y=df_avg['top_10_rank_value'],
                         mode='lines', name='XGB Ranking Portfolio'))

# Update the layout
fig.update_layout(
    title='Portfolio Value Over Time ($1,000 Investment)',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
)

# Show the plot
fig.show()
